### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 1 - Digital Image Processing

In this assignment you will use Python (using Pillow or OpenCV) to load, transcode and store images. You will also use the libraries to extract some basic metadata from the images and store them in a data structure for easy access.

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.


## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=260637#section-13).

If the cell below executes without error, you can start the assignment!

In [3]:
# -------- Imports --------
# Please do not change the contents of this cell!

# In case you work in a local environment on your own machine,
# how to install the required packages:
#   PIP:     pip install <name> / pip install --upgrade <name>
#   CONDA:   conda install -c conda-forge <name>  /  conda install -c anaconda <name>


# Imports required by us.
import cv2                             # opencv-python
from PIL import Image                  # pillow
from PIL.ExifTags import TAGS
from matplotlib import pyplot as plt   # matplotlib
from sklearn.metrics.pairwise import cosine_similarity  # scikit-learn
import numpy as np                     # numpy
import pandas as pd                    # pandas
from IPython.display import display    # packaged with python

# This directive will allow matplotlib to render interactive plots in the notebook.
%matplotlib widget

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [207]:
# Please place your own imports here.
from PIL import ImageFilter, PyAccess, ImageDraw

In [ ]:
# Place any helper functions, global variables and classes here.

## Task 1.1: Image formats transformation and adding filters


In [14]:
# Write your function here.

# Transcodes images to JPEG format of adjustable quality (for JPEG).
def JPEGImageConverter(inputImg: str, outputDir: str, quality: float = 1.0) -> None:
    origName = inputImg.split("/")[-1].split(".")[0]
    Image.open(inputImg).convert('RGB').save(f'{outputDir}/{origName}-{quality}.jpg', 'JPEG', quality=int(quality*100))
    print(f'Image saved at {outputDir}/{origName}-{quality}.jpg')

SyntaxError: unexpected character after line continuation character (1707684414.py, line 8)

In [13]:
# Test your function here.
#JPEGImageConverter("images/b1murene.gif", ".", 0.4)

Image saved at ./b1murene-0.4.jpg


In [23]:
# Add Blur filter
def BlurImage(inputImg: str) -> None:
    origName = inputImg.split("/")[-1].split(".")[0]
    Image.open(inputImg).convert('RGB').filter(ImageFilter.BLUR).save(f'{origName}-blur.jpg')
    print(f'Image saved at {origName}-blur.jpg')
    pass

In [24]:
# Test your function here.
#BlurImage("images/b1murene.gif")

Image saved at b1murene-blur.jpg


## Task 1.2:  Extract / Get Image Metadata

In [142]:
# Write your function here. 

# Extracts metadata from an image and places them into a Pandas DataFrame.
def ImageMetadataGenerator(inputImage: str) -> pd.DataFrame:
    img: Image.Image = Image.open(inputImage)
    return pd.DataFrame(
        data={
            "width":[img.width], 
            "height":[img.height],
            "components":[len(img.convert("RGB").filter(ImageFilter.FIND_EDGES).split())],
            "channels":[len(img.mode)],
            "bits": len(img.getbands()) * 8,
            "colorSpaceType": "".join(img.getbands())
        }
    )

In [152]:
# Test your function here.
#pd.concat([
#    ImageMetadataGenerator("images/b1murene.jpg"),
#    ImageMetadataGenerator("images/b1murene.gif")
#])

,width,height,components,channels,bits,colorSpaceType
0,1862,1241,3,3,24,RGB
0,500,333,3,1,8,P


## Task 1.3: Drawing circles around objects in an image

In [130]:
# Write your function here. 

# Identifies shapes in an images and draws circles around them.
def IdentifyObjects(inputImage: str) -> None:
    
    original = cv2.imread(inputImage)
    image = cv2.GaussianBlur(cv2.cvtColor(original, cv2.COLOR_BGR2GRAY),(3,3),0); #Remove unneccesary details
    image = cv2.Canny(image, 165, 255) # Only need edges, thresholding could replace this line
    
    ############## https://www.geeksforgeeks.org/how-to-detect-shapes-in-images-in-python-using-opencv/ ##############
    
    contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # here we are ignoring first counter because  
    # findcontour function detects whole image as shape 
    for contour in contours[1:]: 
        
        # cv2.approxPloyDP() function to approximate the shape 
        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True) 

        ############## Personal adaptation ##############

        if len(approx) < 10: #Remove false shapes
            continue

        ############## https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html ##############
        
        (x,y),radius = cv2.minEnclosingCircle(contour)
        center = (int(x),int(y))
        radius = int(radius)
        cv2.circle(original,center,radius,(0,10*len(approx),0),2)

        ############## End of extract ##############
    cv2.imshow("",original)
    cv2.waitKey()

In [131]:
# Test your function here.
#IdentifyObjects("images/Task1.3/partyballoons-1.jpg")
#IdentifyObjects("images/Task1.3/partyballoons-2.jpg")

## Task 1.4: Duplicate image finder using cosine similarity

In [229]:
# Calculates a histogram for one image.
def ImageHistogramGenerator(inputImage: str, show=True, forceRGB=False) -> np.array:
    image = Image.open(inputImage)
    if forceRGB:
        image = image.convert('RGB')
    pixelData :PyAccess = image.load()
    width, height = image.size
    #If only one channel {intensity : count}
    if isinstance(pixelData[0,0], int):
        colors = [{}]
        for j in range(256):
                colors[0][j]=0
        for i in range(0, width):
            for j in range(0, height): #10
                colors[0][pixelData[i, j]] += 1
    else: #More than one channel [channel : { intensity : count } ]
        colors = []
        channels = len(image.getbands())
        for i in range(channels):
            colors.append({})
            for j in range(256):
                colors[i][j]=0
        for i in range(0, width): #20
            for j in range(0, height):
                for k in range(channels):
                    colors[k][pixelData[i, j][k]] += 1
    if(show):
        DrawHistogram(image, colors)
    
    return np.array([list(v.values()) for v in colors])
    #30
def DrawHistogram(image: Image.Image, histogram) ->None:
    maxValue = 0
    for i in range(len(histogram)):
        for j in range(255):
            maxValue = max(maxValue, histogram[i][j])
    image = Image.new("".join(image.getbands()), (530, maxValue + 10), color="white")
    draw = ImageDraw.Draw(image)
    pixels = image.load()
    for i in range(len(histogram)):
        color = tuple([255 if x[0] == i else 0 for x in enumerate(range(len(histogram)))]) #Overcomplicated way to get the line colors for the histogram
        for j in range(254):
            draw.line([(j*2+5, maxValue - histogram[i][j]+5),(j*2+6, maxValue - histogram[i][j+1]+5)], fill=color, width=1) #It extracts them from maxValue as the [0,0] it's the top left corner
    image = image.resize((1000, 1000))
    image.show("")

In [230]:
# Test your function here.
#ImageHistogramGenerator("images/b1shell008.jpg", True)

In [233]:
# Find duplicate images and return their similarity scores in a dataframe.
def FindDuplicateImages(inputDir1 : str, inputDir2: str, similarityThreshold: float = 1.0) -> pd.DataFrame:
    
    image1, image2 = (ImageHistogramGenerator(inputDir1, False, True).flatten(), ImageHistogramGenerator(inputDir2, False, True).flatten())
    
    image1, image2 = (image1/np.sum(image1), image2/np.sum(image2))
    
    cosineSimilarity = abs(np.dot(image1,image2)/(np.linalg.norm(image1) * np.linalg.norm(image2)))
    
    return pd.DataFrame(
        data={
            "inputDir1" : [inputDir1],
            "inputDir2" : [inputDir2],
            "similarityThreshold" : [similarityThreshold],
            "result": [cosineSimilarity], 
            "isSame": [cosineSimilarity >= similarityThreshold]
        }
    )

In [234]:
# Test your function here.
"""pd.concat([
    FindDuplicateImages("images/b17maartent1427.jpg","images/b17maartent1427.jpg", 0.7),
    FindDuplicateImages("images/b17maartent1427.gif","images/b17maartent1427.gif", 0.7),
    FindDuplicateImages("images/b17maartent1427.gif","images/b17maartent1427.jpg", 0.7),
    FindDuplicateImages("images/b17barb026.jpg","images/b17barb026_dithered.gif", 0.7),
    FindDuplicateImages("images/b17barb026.jpg","images/b1murene.jpg", 0.7),
    FindDuplicateImages("images/T1.4/b1shell008.jpg","images/T1.4/b1shell008-1.jpg", 0.7),
    FindDuplicateImages("images/T1.4/jawa-2.1.jpg","images/T1.4/jawa-2.jpg", 0.7),
    FindDuplicateImages("images/T1.4/m-4.1.jpg","images/T1.4/m-4.jpg", 0.7),
])"""

,inputDir1,inputDir2,similarityThreshold,result,isSame
0,images/b17maartent1427.jpg,images/b17maartent1427.jpg,0.7,1.000000,True
0,images/b17maartent1427.gif,images/b17maartent1427.gif,0.7,1.000000,True
0,images/b17maartent1427.gif,images/b17maartent1427.jpg,0.7,0.267831,False
0,images/b17barb026.jpg,images/b17barb026_dithered.gif,0.7,0.273144,False
0,images/b17barb026.jpg,images/b1murene.jpg,0.7,0.370309,False
0,images/T1.4/b1shell008.jpg,images/T1.4/b1shell008-1.jpg,0.7,0.999717,True
0,images/T1.4/jawa-2.1.jpg,images/T1.4/jawa-2.jpg,0.7,0.962974,True
0,images/T1.4/m-4.1.jpg,images/T1.4/m-4.jpg,0.7,0.998856,True
